# 🏠 Gayrimenkul Tahmincisi

🎯Aşağıdaki challenge'da şunlar arasında bir ***DOĞRUSAL İLİŞKİ*** olup olmadığını anlamaya çalışacağız:
- bir dairenin **fiyatı** (her daire için **hedefimiz**)
- ve yüzey alanı, yatak odası sayısı vb. gibi **özellikler**

❗️ Bu challenge'da Pandas yasaklanmıştır ❗️

😁 Bu alıştırma boyunca arkadaşın olacak [NumPy belgelerine](https://docs.scipy.org/doc/numpy/reference/) hoş geldin.

👉 Ayrıca bu [NumPy cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf) adresinde yardım bulabilirsin

In [2]:
import numpy as np


🌆 Aşağıda 4 dairenin verilerini topladığımızı varsay: 
- **özellikleri**:
    - `surface` (kare feet)
    - `bedrooms`
    - `floors` 
- **hedefleri**:
    - `price` (bin USD cinsinden)

|daireler |surface (kare feet)|yatak odası|katlar|price (bin USD)|
|------|-------------|--------|------|------------|
|flat1 |620|1|1|244|
|flat2 |3280|4|2|671|
|flat3 |1900|2|2|504|
|flat4 |1320|3|3|510|

👉 **Bir dairenin fiyatını tahmin etmenin** ilk yaklaşımı **hedef ve özellikler arasında doğrusal bir ilişki bulmaya** çalışmaktır (*yani fiyat ile (yüzey, yatak odası, kat) arasında*), aşağıdaki **$n = 4$ doğrusal denklem sistemi ile $p = 4$ bilinmeyen değişken** çözerek:

$$\begin{cases}
    244 = \theta_0 + 620\theta_1 + 1\theta_2 + 1\theta_3 \\
    671 = \theta_0 + 3280\theta_1 + 4\theta_2 + 2\theta_3 \\
    504 = \theta_0 + 1900\theta_1 + 2\theta_2 + 2\theta_3 \\
    510 = \theta_0 + 1320\theta_1 + 3\theta_2 + 3\theta_3 \\
\end{cases}$$

bu da bir matris denklemine çevrilebilir:

$$\boldsymbol y = \boldsymbol {X \cdot \theta}$$

$$\begin{bmatrix}
    244 \\
    671 \\
    504 \\
    510
\end{bmatrix}_{4 \times 1} = \begin{bmatrix}
    1 & 620 & 1 & 1 \\
    1 & 3280 & 4 & 2 \\
    1 & 1900 & 2 & 2 \\
    1 & 1320 & 3 & 3
\end{bmatrix}_{4 \times 4} \begin{bmatrix}
    \theta_0 \\
    \theta_1 \\
    \theta_2 \\
    \theta_3
\end{bmatrix}_{4 \times 1}$$

burada:
* $\boldsymbol y$ **`hedef`**, `Price` vektörüdür
* $\boldsymbol X$ **`özelliklerin matrisi`** temsil eder
* $\boldsymbol {\theta} = \begin{bmatrix}
    \theta_0 \\
    \theta_1 \\
    \theta_2 \\
    \theta_3
\end{bmatrix}$ (*theta*) bulunacak **katsayıları/değişkenleri/bilinmeyenleri** temsil eden vektördür

----

🤓 Burada, **özelliklerimizin** katsayılarını temsil etmek için Yunanca harfi `theta` $\boldsymbol \theta = \begin{bmatrix}
    \theta_0 \\
    \theta_1 \\
    \theta_2 \\
    \theta_3 \\
\end{bmatrix}$ kullanıyoruz:

- Yüzeyi, yatak odası ve katı olmayan bir daire $\theta_0$ kadar mal olur
- Bir kare ayağın artması - *yatak odası sayısı ve kat numarası sabit tutulduğunda* - fiyatı $\theta_1$ bin USD kadar artırır
- Ek bir yatak odası - *yüzey ve kat numarası sabit tutulduğunda* - fiyatı $\theta_2$ bin USD kadar artırır
- Bir kat numarasının artması - *yüzey ve yatak odası sayısı sabit tutulduğunda* - fiyatı $\theta_3$ bin USD kadar artırır

----

😉 Bu doğrusal denklem sistemini çözmeyi başarırsak (yani $\theta_0$, $\theta_1$, $\theta_2$, $\theta_3$ bulursak), herhangi bir yeni dairenin fiyatı aşağıdaki formül kullanılarak tahmin edilebilir: $$y_{newflat} = \boldsymbol x_{newflat} \cdot \boldsymbol \theta$$

## (1) `features` matrisini $\boldsymbol X$ tanımla:

❓ 4 gözlem için 3 özelliğin (yüzey, yatak odası, katlar) değerlerini depolayan bir $(4,3)$ `numpy.ndarray` oluştur.

In [3]:
X = np.array([
    [620,1,1],
    [3280,4,2],
    [1900,2,2],
    [1320,3,3]])
X

array([[ 620,    1,    1],
       [3280,    4,    2],
       [1900,    2,    2],
       [1320,    3,    3]])

❓ Bu ***np.array***'in ***shape***'ini, ***size***'ını ve ***dim***'ini double-check et

In [4]:
print(X.shape)
print(X.ndim)
print(X.dtype)

(4, 3)
2
int64


❓ Doğrusal denklem sistemini temsil eden $(4,4)$ matrisi $\boldsymbol X$ oluşturmak için $ = \begin{bmatrix}
    1 \\
    1 \\
    1 \\
    1 \\
\end{bmatrix}$ birlerinin "constant" vektörünü ekle

<details>
    <summary><i>Açıklamalar</i></summary>

🤔 Muhtemelen fark ettiğin gibi, doğrusal denklem sistemi 4 denklemde görünen bir $\theta_0$ katsayısını içeriyor.

❗️ Doğrusal regresyon çizgisinin y-kesişimini temsil etmek için ek bir özelliğe ihtiyacımız var

_Not_ : `price` ve özellikler arasında sıkı bir doğrusal ilişki yerine bir [affine relation](https://math.stackexchange.com/questions/275310/what-is-the-difference-between-linear-and-affine-function) hakkında konuşuyoruz (_Bkz. Decision Science Module_)
    
</details>

In [5]:
# x0'ı en hızlı NumPy yöntemi ile 1 ile doldurulmuş bir (4,1) vektörü olarak tanımla
x0 = np.ones((4,1))
x0

array([[1.],
       [1.],
       [1.],
       [1.]])

In [6]:
# x0'ı önceki (4,3) matrisinle birleştirerek (4,4) matris X oluşturmak için `numpy.hstack` kullan
X = np.hstack((x0,X))
X

array([[1.00e+00, 6.20e+02, 1.00e+00, 1.00e+00],
       [1.00e+00, 3.28e+03, 4.00e+00, 2.00e+00],
       [1.00e+00, 1.90e+03, 2.00e+00, 2.00e+00],
       [1.00e+00, 1.32e+03, 3.00e+00, 3.00e+00]])

**🧪 Kodunu kontrol et!**

In [7]:
from nbresult import ChallengeResult

result = ChallengeResult('features',
    features=X
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/tumay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/tumay/code/data_real_estate_estimator/tests
plugins: anyio-4.8.0, dash-3.3.0, typeguard-4.4.2
collecting ... collected 3 items

test_features.py::TestFeatures::test_X_shape PASSED                      [ 33%]
test_features.py::TestFeatures::test_features_order PASSED               [ 66%]
test_features.py::TestFeatures::test_x0_is_ones PASSED                   [100%]

============================== 3 passed in 0.17s ===============================


💯 You can commit your code:

git add tests/features.pickle

git commit -m 'Completed features step'

git push origin master



## (2) `Prices`'ın $\boldsymbol y$ vektörünü tanımla

$\boldsymbol y  = \begin{bmatrix}
    244 \\
    671 \\
    504 \\
    510
\end{bmatrix}$

❓ Matris gösterimimiz $\boldsymbol y  = \boldsymbol {X\cdot \theta}$ ile eşleştirmek için $\boldsymbol y$'nin shape'i ne olmalı? Aşağıda $\boldsymbol y$'yi tanımla. ❓

<details>
    <summary><i>İpucu</i></summary>

$\boldsymbol y$ bir $(4,1)$ array olmalı, dikey olarak temsil edilen düz bir "vektör" eşdeğeri
</details>

In [8]:
y = np.array([
    [244],
    [671],
    [504],
    [510]])
y.shape

(4, 1)

**🧪 Kodunu kontrol et!**

In [9]:
from nbresult import ChallengeResult

result = ChallengeResult('target',
    target=y
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/tumay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/tumay/code/data_real_estate_estimator/tests
plugins: anyio-4.8.0, dash-3.3.0, typeguard-4.4.2
collecting ... collected 1 item

test_target.py::TestTarget::test_shape PASSED                            [100%]

============================== 1 passed in 0.11s ===============================


💯 You can commit your code:

git add tests/target.pickle

git commit -m 'Completed target step'

git push origin master



## (3) Sistemin çözümünü bul

⏰Şimdi katsayıların $\boldsymbol \theta = \begin{bmatrix}
    \theta_0 \\
    \theta_1 \\
    \theta_2 \\
    \theta_3
\end{bmatrix}$ vektörünü bulmanın zamanı!

👍 Denklemin çözümü:
 
$$ \large \boldsymbol X \cdot \boldsymbol \theta = \boldsymbol y 
\large \iff \boldsymbol X^{-1} \cdot \boldsymbol X \boldsymbol \cdot \theta = \boldsymbol X^{-1} \cdot \boldsymbol y 
\large \iff \boldsymbol \theta = \boldsymbol X^{-1} \cdot \boldsymbol y
$$

burada $\large \boldsymbol X^{-1}$, $\large \boldsymbol X$'in tersidir.

In [ ]:
# Doğru NumPy yöntemi ile X matrisinin tersini hesapla

👉 Aşağıdaki eşitliği test ederek tersinin çalışıp çalışmadığını kontrol edebilirsin:

$$\boldsymbol X^{-1} \cdot\boldsymbol X = \boldsymbol I_4$$
burada $\boldsymbol I_4$ $ 4 \times 4 $ birim matrisidir $ \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1
\end{bmatrix}$

In [ ]:
# Doğru NumPy yöntemi kullanarak I4'ü tanımla

👉 Şimdi $\boldsymbol X^{-1} \boldsymbol X$'i hesapla:

In [ ]:
# YOUR CODE HERE

❓ $\boldsymbol I_4$'e benziyor mu? ❓

⛔️ Benzemiyorsa, muhtemelen $\boldsymbol X^{-1}$ ve $\boldsymbol X$ arasındaki çarpımı gerçekleştirmek için `*` operatörünü kullandın. Burada matris çarpımını yapmak istiyoruz. Bunu yapmak için doğru NumPy yöntemini (veya kısaltılmış operatörü) bul.

✅ Benziyorsa, elde edilen çarpımda tam olarak sıfır ve birler elde edemediğini fark etmiş olabilirsin. Emin olmak için sonucunu kontrol etmek amacıyla [`numpy.allclose()`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html?highlight=allclose#numpy.allclose) yöntemini dene:

In [ ]:
# YOUR CODE HERE

🎉 Son olarak aşağıdaki formülü kullanarak `theta`'yı hesaplayabilirsin: $ \large \boldsymbol \theta = \boldsymbol X^{-1}\cdot \boldsymbol y $:

In [ ]:
# YOUR CODE HERE

**🧪 Kodunu kontrol et!**

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('solution',
    theta=theta
)
result.write()
print(result.check())

## (4) Yeni bir dairenin fiyatının tahmini

Sonunda sistemi çözerek $\boldsymbol \theta$'yı buldun ve şimdi bu özelliklere sahip 5. bir dairenin `Price`'ını (bin $ cinsinden) tahmin edebilirsin:

- `Surface`: 3000 $ft^2$
- `Bedrooms`: 5 
- `Floors`: 1

aşağıdaki formül ile:

$$y_{flat5} = \boldsymbol x_{flat5} \cdot \boldsymbol \theta$$

In [ ]:
# x5'i tanımla



# y5'i hesapla
# 526.000 $ fiyatını bulmalısın

## (5) Gerçeklik-kontrol

❗️ Gerçekte, bir dairenin fiyatı hiçbir zaman tamamen yüzey, yatak odası sayısı ve kat numarası tarafından belirlenmez.

Gerçek $y_{flat5}$ fiyatını tahmin edilen $526.000$ yerine $700.000$ olarak ölçtüğümüzü hayal edelim.

🤨 Modelimizi geliştirmek için bu yeni bilgiyi dikkate alabilir miyiz?

❓ Bu yeni dairenin bilgisini içermek için $ \large \boldsymbol X \cdot \boldsymbol \theta = \boldsymbol y$ doğrusal denklem sistemini güncelle.

In [ ]:
# Shape (5,4)'ün yeni özellikleri matrisini X oluştur
# Shape'i (5,4) olduğundan emin olmak için yazdır

In [ ]:
# Shape (5,1) olan yeni y oluştur

👉 Güncellenmiş modelimizden 6. bir dairenin fiyatını tahmin etmeyi deneyelim.
Bunu yapmak için [`numpy.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html) kullanarak $\boldsymbol X \cdot \boldsymbol \theta = \boldsymbol y$'dan $\boldsymbol \theta$'yı çözmeyi dene.

😲 Ne oluyor? Neden hata aldığını anlamaya çalış. Sonra, hücrenin altındaki açıklamayı aç.

In [ ]:
# YOUR CODE HERE

<details>
    <summary><i>Açıklamalar</i></summary>

$ \large \boldsymbol X$ kare bir matris değil

$ \large  \rightarrow$ bu nedenle tersinin alınamaz: $ \large  \boldsymbol X^{-1}$ mevcut değil
 
$ \large  \rightarrow$ $ \large \boldsymbol \theta$, $ \large  \boldsymbol y = \boldsymbol X \cdot \boldsymbol \theta$'dan hesaplanamaz
    
😕 Sadece 3 özelliğin doğrusal kombinasyonu olarak bir dairenin tahmin edilen fiyatını hesaplamak için kapalı bir matematiksel formül bulmaktan oluşan ilk yaklaşımımız, gözlemlenen 5 dairemiz için **geçerli değildir**.

😉 ***Sürece güven!*** 

$ \large  \rightarrow$ Bunun yerine, gelecek haftalarda bu özellikler temelinde bir dairenin fiyatını **yaklaşıklaştırmak** için yöntemler öğreneceğiz.

Örneğin, $\large  \boldsymbol y = \boldsymbol X \cdot \boldsymbol \theta$ çözmek yerine, hata $ \large \\boldsymbol e = \\boldsymbol X \\cdot \\hat{\\boldsymbol \\theta} - \\boldsymbol y $'ü en aza indiren $ \large  \\hat{\\boldsymbol \\theta}$'yı bulabiliriz: Bu yaklaşım **Doğrusal Regresyon modeli** olarak adlandırılır

Bu yeni tahminci daha sonra $ \large  \\hat y_{flat_6} = \\boldsymbol x_{flat_6} \\cdot \\hat{\\boldsymbol \\theta}$ ile herhangi bir yeni daire için fiyatın **yaklaşık** bir tahminini vermek için kullanılabilir

</details>

🏁 Tebrikler!

💾 Sonraki challenge'a geçmeden önce notebook'u commit etmeyi ve push'lamayı unutma!